In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
k = 64
tree_depth = 10
batch_size = 512
device = 'cpu'
train_data_path = r'<>/mitbih_train.csv'  # replace <> with the correct path of the dataset
test_data_path = r'<>/mitbih_test.csv'  # replace <> with the correct path of the dataset

In [9]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [10]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [11]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [12]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [13]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

/home/eitan.k/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.611621379852295 | KNN Loss: 5.851576805114746 | CLS Loss: 1.7600445747375488
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 5.5411458015441895 | KNN Loss: 4.718804836273193 | CLS Loss: 0.8223410844802856
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 5.31278133392334 | KNN Loss: 4.556290626525879 | CLS Loss: 0.7564908266067505
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 5.130352973937988 | KNN Loss: 4.530723571777344 | CLS Loss: 0.5996293425559998
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 5.090763092041016 | KNN Loss: 4.479553699493408 | CLS Loss: 0.6112095713615417
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 5.027774810791016 | KNN Loss: 4.468639850616455 | CLS Loss: 0.5591350793838501
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 4.985223770141602 | KNN Loss: 4.48076868057251 | CLS Loss: 0.504455029964447
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 4.821516990661621 | KNN Loss: 4.4435038566589355 | CLS Los

Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 4.386694431304932 | KNN Loss: 4.247145175933838 | CLS Loss: 0.1395494043827057
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 4.374841690063477 | KNN Loss: 4.246660232543945 | CLS Loss: 0.12818162143230438
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 4.411810874938965 | KNN Loss: 4.282763481140137 | CLS Loss: 0.12904725968837738
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 4.402511119842529 | KNN Loss: 4.281371116638184 | CLS Loss: 0.12113998830318451
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 4.4582953453063965 | KNN Loss: 4.3242268562316895 | CLS Loss: 0.13406848907470703
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 4.4174981117248535 | KNN Loss: 4.270980358123779 | CLS Loss: 0.14651760458946228
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 4.4136857986450195 | KNN Loss: 4.308887958526611 | CLS Loss: 0.10479802638292313
Epoch 4 / 200 | iteration 160 / 171 | Total Loss: 4.330372333526611 | KNN Loss: 4.249135

Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 4.397749900817871 | KNN Loss: 4.290170669555664 | CLS Loss: 0.10757926106452942
Epoch: 007, Loss: 4.3257, Train: 0.9791, Valid: 0.9756, Best: 0.9756
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 4.312385082244873 | KNN Loss: 4.230754852294922 | CLS Loss: 0.08163004368543625
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 4.3073225021362305 | KNN Loss: 4.232346534729004 | CLS Loss: 0.07497601956129074
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 4.376765727996826 | KNN Loss: 4.256069183349609 | CLS Loss: 0.12069671601057053
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 4.250857830047607 | KNN Loss: 4.192142486572266 | CLS Loss: 0.0587153285741806
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 4.308696269989014 | KNN Loss: 4.215068340301514 | CLS Loss: 0.09362789988517761
Epoch 8 / 200 | iteration 50 / 171 | Total Loss: 4.2926836013793945 | KNN Loss: 4.228835582733154 | CLS Loss: 0.06384791433811188
Epoch 8 / 200 | iteration 6

Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 4.251927375793457 | KNN Loss: 4.207891941070557 | CLS Loss: 0.044035378843545914
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 4.266667366027832 | KNN Loss: 4.218547344207764 | CLS Loss: 0.048120222985744476
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 4.2895026206970215 | KNN Loss: 4.229923248291016 | CLS Loss: 0.05957914888858795
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 4.284947395324707 | KNN Loss: 4.229155540466309 | CLS Loss: 0.05579190328717232
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 4.2707695960998535 | KNN Loss: 4.1840410232543945 | CLS Loss: 0.08672859519720078
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 4.274397373199463 | KNN Loss: 4.226527690887451 | CLS Loss: 0.04786955937743187
Epoch 11 / 200 | iteration 130 / 171 | Total Loss: 4.230456829071045 | KNN Loss: 4.206214427947998 | CLS Loss: 0.024242430925369263
Epoch 11 / 200 | iteration 140 / 171 | Total Loss: 4.3067755699157715 | KNN Loss

...

Epoch 79 / 200 | iteration 90 / 171 | Total Loss: 4.150318622589111 | KNN Loss: 4.1280107498168945 | CLS Loss: 0.022307857871055603
Epoch 79 / 200 | iteration 100 / 171 | Total Loss: 4.149371147155762 | KNN Loss: 4.111486911773682 | CLS Loss: 0.03788416460156441
Epoch 79 / 200 | iteration 110 / 171 | Total Loss: 4.168167591094971 | KNN Loss: 4.155006408691406 | CLS Loss: 0.013161257840692997
Epoch 79 / 200 | iteration 120 / 171 | Total Loss: 4.1702680587768555 | KNN Loss: 4.153422832489014 | CLS Loss: 0.01684541627764702
Epoch 79 / 200 | iteration 130 / 171 | Total Loss: 4.148903846740723 | KNN Loss: 4.144194602966309 | CLS Loss: 0.004709448665380478
Epoch 79 / 200 | iteration 140 / 171 | Total Loss: 4.147111415863037 | KNN Loss: 4.144194602966309 | CLS Loss: 0.0029165949672460556
Epoch 79 / 200 | iteration 150 / 171 | Total Loss: 4.185890197753906 | KNN Loss: 4.164979934692383 | CLS Loss: 0.02091006375849247
Epoch 79 / 200 | iteration 160 / 171 | Total Loss: 4.15160608291626 | KNN Los

Epoch 82 / 200 | iteration 160 / 171 | Total Loss: 4.130731582641602 | KNN Loss: 4.118416786193848 | CLS Loss: 0.012314657680690289
Epoch 82 / 200 | iteration 170 / 171 | Total Loss: 4.144529342651367 | KNN Loss: 4.137655735015869 | CLS Loss: 0.006873531267046928
Epoch: 082, Loss: 4.1675, Train: 0.9973, Valid: 0.9870, Best: 0.9879
Epoch 83 / 200 | iteration 0 / 171 | Total Loss: 4.1322736740112305 | KNN Loss: 4.130181312561035 | CLS Loss: 0.0020925994031131268
Epoch 83 / 200 | iteration 10 / 171 | Total Loss: 4.155758857727051 | KNN Loss: 4.1355695724487305 | CLS Loss: 0.020189112052321434
Epoch 83 / 200 | iteration 20 / 171 | Total Loss: 4.145876884460449 | KNN Loss: 4.139819622039795 | CLS Loss: 0.006057152524590492
Epoch 83 / 200 | iteration 30 / 171 | Total Loss: 4.153669834136963 | KNN Loss: 4.14516544342041 | CLS Loss: 0.008504399098455906
Epoch 83 / 200 | iteration 40 / 171 | Total Loss: 4.1693220138549805 | KNN Loss: 4.161662578582764 | CLS Loss: 0.007659585680812597
Epoch 83 /

Epoch 86 / 200 | iteration 50 / 171 | Total Loss: 4.142706394195557 | KNN Loss: 4.13645601272583 | CLS Loss: 0.006250463426113129
Epoch 86 / 200 | iteration 60 / 171 | Total Loss: 4.159805774688721 | KNN Loss: 4.154379844665527 | CLS Loss: 0.005426023155450821
Epoch 86 / 200 | iteration 70 / 171 | Total Loss: 4.187921047210693 | KNN Loss: 4.169433116912842 | CLS Loss: 0.018488062545657158
Epoch 86 / 200 | iteration 80 / 171 | Total Loss: 4.153057098388672 | KNN Loss: 4.150600433349609 | CLS Loss: 0.0024564729537814856
Epoch 86 / 200 | iteration 90 / 171 | Total Loss: 4.174063682556152 | KNN Loss: 4.159762859344482 | CLS Loss: 0.014300926588475704
Epoch 86 / 200 | iteration 100 / 171 | Total Loss: 4.151689052581787 | KNN Loss: 4.1401824951171875 | CLS Loss: 0.011506634764373302
Epoch 86 / 200 | iteration 110 / 171 | Total Loss: 4.157598972320557 | KNN Loss: 4.133328914642334 | CLS Loss: 0.024270083755254745
Epoch 86 / 200 | iteration 120 / 171 | Total Loss: 4.167727947235107 | KNN Loss:

Epoch 89 / 200 | iteration 120 / 171 | Total Loss: 4.14461088180542 | KNN Loss: 4.140966892242432 | CLS Loss: 0.003643893403932452
Epoch 89 / 200 | iteration 130 / 171 | Total Loss: 4.188840389251709 | KNN Loss: 4.163620471954346 | CLS Loss: 0.02521994151175022
Epoch 89 / 200 | iteration 140 / 171 | Total Loss: 4.14785099029541 | KNN Loss: 4.139376163482666 | CLS Loss: 0.008474699221551418
Epoch 89 / 200 | iteration 150 / 171 | Total Loss: 4.1736555099487305 | KNN Loss: 4.167597770690918 | CLS Loss: 0.006057725753635168
Epoch 89 / 200 | iteration 160 / 171 | Total Loss: 4.142537593841553 | KNN Loss: 4.139855861663818 | CLS Loss: 0.0026817191392183304
Epoch 89 / 200 | iteration 170 / 171 | Total Loss: 4.176389217376709 | KNN Loss: 4.148622512817383 | CLS Loss: 0.027766840532422066
Epoch: 089, Loss: 4.1597, Train: 0.9971, Valid: 0.9867, Best: 0.9879
Epoch 90 / 200 | iteration 0 / 171 | Total Loss: 4.143197059631348 | KNN Loss: 4.127331733703613 | CLS Loss: 0.015865514054894447
Epoch 90 /

KeyboardInterrupt: 

In [14]:
test(model, test_data_iter, device)

tensor(0.9865)

In [15]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

  0%|          | 0/43 [00:00<?, ?it/s]

In [18]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [20]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

Number of inliers: 0.9410716732903933


In [21]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [22]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [23]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [24]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [25]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [26]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [27]:
losses = []
accs = []
sparsity = []

In [ ]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
Epoch: 00 | Batch: 000 / 041 | Total loss: 1.587 | Reg loss: 0.012 | Tree loss: 1.587 | Accuracy: 0.691406 | 1.298 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 01 | Batch: 000 / 041 | Total loss: 1.455 | Reg loss: 0.005 | Tree loss: 1.455 | Accuracy: 0.677734 | 2.015 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 02 | Batch: 000 / 041 

Epoch: 20 | Batch: 000 / 041 | Total loss: 0.872 | Reg loss: 0.021 | Tree loss: 0.872 | Accuracy: 0.695312 | 1.863 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 21 | Batch: 000 / 041 | Total loss: 0.838 | Reg loss: 0.021 | Tree loss: 0.838 | Accuracy: 0.708984 | 1.864 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 22 | Batch: 000 / 041 | Total loss: 0.850 | Reg loss: 0.021 | Tree loss: 0.850 | Accuracy: 0.705078 | 1.863 sec/iter
Average sparseness: 0.9840425531914894
layer 0

Epoch: 40 | Batch: 000 / 041 | Total loss: 0.840 | Reg loss: 0.023 | Tree loss: 0.840 | Accuracy: 0.708984 | 1.853 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 41 | Batch: 000 / 041 | Total loss: 0.921 | Reg loss: 0.023 | Tree loss: 0.921 | Accuracy: 0.681641 | 1.854 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 42 | Batch: 000 / 041 | Total loss: 0.921 | Reg loss: 0.023 | Tree loss: 0.921 | Accuracy: 0.691406 | 1.852 sec/iter
Average sparseness: 0.9840425531914894
layer 0

Epoch: 60 | Batch: 000 / 041 | Total loss: 0.710 | Reg loss: 0.024 | Tree loss: 0.710 | Accuracy: 0.748047 | 1.85 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 61 | Batch: 000 / 041 | Total loss: 0.805 | Reg loss: 0.024 | Tree loss: 0.805 | Accuracy: 0.708984 | 1.853 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 62 | Batch: 000 / 041 | Total loss: 0.702 | Reg loss: 0.024 | Tree loss: 0.702 | Accuracy: 0.740234 | 1.856 sec/iter
Average sparseness: 0.9840425531914894
layer 0:

Epoch: 80 | Batch: 000 / 041 | Total loss: 0.671 | Reg loss: 0.024 | Tree loss: 0.671 | Accuracy: 0.765625 | 1.878 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 81 | Batch: 000 / 041 | Total loss: 0.725 | Reg loss: 0.024 | Tree loss: 0.725 | Accuracy: 0.730469 | 1.878 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 82 | Batch: 000 / 041 | Total loss: 0.717 | Reg loss: 0.024 | Tree loss: 0.717 | Accuracy: 0.736328 | 1.881 sec/iter
Average sparseness: 0.9840425531914894
layer 0

Epoch: 100 | Batch: 000 / 041 | Total loss: 0.789 | Reg loss: 0.025 | Tree loss: 0.789 | Accuracy: 0.726562 | 1.89 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 101 | Batch: 000 / 041 | Total loss: 0.755 | Reg loss: 0.025 | Tree loss: 0.755 | Accuracy: 0.705078 | 1.888 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 102 | Batch: 000 / 041 | Total loss: 0.692 | Reg loss: 0.025 | Tree loss: 0.692 | Accuracy: 0.714844 | 1.89 sec/iter
Average sparseness: 0.9840425531914894
layer 

Epoch: 120 | Batch: 000 / 041 | Total loss: 0.700 | Reg loss: 0.025 | Tree loss: 0.700 | Accuracy: 0.742188 | 1.894 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 121 | Batch: 000 / 041 | Total loss: 0.782 | Reg loss: 0.025 | Tree loss: 0.782 | Accuracy: 0.693359 | 1.894 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 122 | Batch: 000 / 041 | Total loss: 0.832 | Reg loss: 0.025 | Tree loss: 0.832 | Accuracy: 0.673828 | 1.897 sec/iter
Average sparseness: 0.9840425531914894
laye

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
print(f"Number of patterns: {len(root.get_leaves())}")

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")